In [3]:
%%bash
kind create cluster --config kind-config.yaml

Creating cluster "kind" ...
 • Ensuring node image (kindest/node:v1.24.0) 🖼  ...
 ✓ Ensuring node image (kindest/node:v1.24.0) 🖼
 • Preparing nodes 📦   ...
 ✓ Preparing nodes 📦 
 • Writing configuration 📜  ...
 ✓ Writing configuration 📜
 • Starting control-plane 🕹️  ...
 ✓ Starting control-plane 🕹️
 • Installing CNI 🔌  ...
 ✓ Installing CNI 🔌
 • Installing StorageClass 💾  ...
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Thanks for using kind! 😊


In [5]:
%%bash
kubectl get nodes

NAME                 STATUS   ROLES           AGE   VERSION
kind-control-plane   Ready    control-plane   78s   v1.24.0


In [4]:
%%bash
kubectl create namespace argo
kubectl apply -n argo -f https://github.com/argoproj/argo-workflows/releases/download/v3.3.9/install.yaml

namespace/argo created
customresourcedefinition.apiextensions.k8s.io/clusterworkflowtemplates.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/cronworkflows.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workfloweventbindings.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workflows.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workflowtaskresults.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workflowtasksets.argoproj.io created
customresourcedefinition.apiextensions.k8s.io/workflowtemplates.argoproj.io created
serviceaccount/argo created
serviceaccount/argo-server created
role.rbac.authorization.k8s.io/argo-role created
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-admin created
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-edit created
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-view created
clusterrole.rbac.authorization.k8s.io/argo-cluster-role created
clust

See: https://argoproj.github.io/argo-workflows/quick-start/

In [18]:
%%bash
kubectl patch deployment \
  argo-server \
  --namespace argo \
  --type='json' \
  -p='[{"op": "replace", "path": "/spec/template/spec/containers/0/args", "value": [
  "server",
  "--auth-mode=server"
]}]'

deployment.apps/argo-server patched


```sh
kubectl -n argo port-forward deployment/argo-server 2746:2746
```

In [7]:
%%bash
kubectl get pods -n argo

NAME                                   READY   STATUS    RESTARTS   AGE
argo-server-65566599f8-hpjbb           1/1     Running   0          24s
workflow-controller-77c44779bf-8mrvt   1/1     Running   0          24s


In [11]:
%%bash
pip install ploomber soopervisor

In [13]:
%%bash
ploomber examples -n templates/ml-intermediate -o ml-intermediate
cd ml-intermediate

# configure development environment
cp requirements.txt requirements.lock.txt
pip install -r requirements.txt

Loading examples...
There's a new ploomber version available (0.21), you're running 0.21.1dev. To upgrade: pip install ploomber --upgrade
======== Copying example templates/ml-intermediate to ml-intermediate/ =========
Next steps:

$ cd ml-intermediate/
$ ploomber install

Open ml-intermediate/README.md for details.
  Using cached ipywidgets-8.0.1-py3-none-any.whl (133 kB)
  Using cached pyarrow-9.0.0-cp39-cp39-macosx_10_13_x86_64.whl (24.0 MB)
  Using cached sklearn_evaluation-0.7-py3-none-any.whl (46 kB)
  Using cached widgetsnbextension-4.0.2-py3-none-any.whl (2.0 MB)
  Using cached jupyterlab_widgets-3.0.2-py3-none-any.whl (383 kB)
  Using cached black-22.6.0-cp39-cp39-macosx_10_9_x86_64.whl (1.4 MB)
  Using cached matplotlib-3.5.3-cp39-cp39-macosx_10_9_x86_64.whl (7.3 MB)
  Using cached pathspec-0.9.0-py2.py3-none-any.whl (31 kB)
  Using cached mypy_extensions-0.4.3-py2.py3-none-any.whl (4.5 kB)
  Using cached platformdirs-2.5.2-py3-none-any.whl (14 kB)
  Using cached kiwisolver-1

In [15]:
%%bash
cd ml-intermediate
soopervisor add training --backend argo-workflows

================================= Loading DAG ==================================
No pipeline.training.yaml found, looking for pipeline.yaml instead
Found /Users/Edu/dev/soopervisor/kind/ml-intermediate/pipeline.yaml. Loading...
= Adding /Users/Edu/dev/soopervisor/kind/ml-intermediate/training/Dockerfile... =
===================================== Done =====================================
Environment added, to export it:
	 $ soopervisor export training
To force execution of all tasks:
	 $ soopervisor export training --mode force



In [16]:
%%bash
cd ml-intermediate
curl https://raw.githubusercontent.com/ploomber/soopervisor/master/tutorials/kubernetes/env-k8s.yaml -o env.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    38  100    38    0     0    208      0 --:--:-- --:--:-- --:--:--   208


In [17]:
%%bash
cd ml-intermediate
curl https://raw.githubusercontent.com/ploomber/soopervisor/master/tutorials/kubernetes/soopervisor-k8s.yaml -o soopervisor.yaml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   284  100   284    0     0   1007      0 --:--:-- --:--:-- --:--:--  1007


In [37]:
%%bash
cd ml-intermediate
soopervisor export training --skip-tests --ignore-git --mode force

================================= Loading DAG ==================================
No pipeline.training.yaml found, looking for pipeline.yaml instead
Found /Users/Edu/dev/soopervisor/kind/ml-intermediate/pipeline.yaml. Loading...
================================= Loading DAG ==================================
No pipeline.training.yaml found, looking for pipeline.yaml instead
Found /Users/Edu/dev/soopervisor/kind/ml-intermediate/pipeline.yaml. Loading...
================================ Packaging code ================================
Copying soopervisor.yaml -> dist/ml-intermediate/soopervisor.yaml
Copying env.serve.yaml -> dist/ml-intermediate/env.serve.yaml
Copying tasks/__init__.py -> dist/ml-intermediate/tasks/__init__.py
Copying tasks/serve.py -> dist/ml-intermediate/tasks/serve.py
Copying tasks/features.py -> dist/ml-intermediate/tasks/features.py
Copying tasks/join.py -> dist/ml-intermediate/tasks/join.py
Copying tasks/get.py -> dist/ml-intermediate/tasks/get.py
Copying fit.py -> d

100%|██████████| 5/5 [00:00<00:00, 3272.20it/s]
#1 [internal] load build definition from Dockerfile
#1 sha256:f89d895001ec67b441260e7a18d9becac6e2cfcbbe0968174a4beb2593e446a4
#1 transferring dockerfile: 37B 0.0s done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 sha256:e03270e169da127e3d41269228afb3ac1ce7456ceeb95cc669762193447e3c2f
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/condaforge/mambaforge:4.10.1-0
#3 sha256:9b12efa2a5d145f1e49b558cab6574b00e33d523e33b5f48656f7ff13e753cb9
#3 DONE 1.7s

#4 [1/7] FROM docker.io/condaforge/mambaforge:4.10.1-0@sha256:0afe653d5d0af8269e221009c88f378b4d5662967114d91de6b38171985d9637
#4 sha256:f54afc42d2bd43672081f35d5c21b3b7903983738735bf8fed474deb9617d9bb
#4 DONE 0.0s

#5 [internal] load build context
#5 sha256:5b00ba47c695241a2ac83a49666dbda9b7edfa2cc6a48d021c8706829d85a285
#5 transferring context: 127.75kB 0.0s done
#5 DONE 0.0s

#6 [2/7] COPY requirements.lock.txt project/requirements.lock.txt
#6 s

In [38]:
%%bash
kind load docker-image ml-intermediate:latest-default

Image: "ml-intermediate:latest-default" with ID "sha256:6ffb204a32c6c71f298bfeaa11002fdb9d49b9e6119daf00cd68cbef29f21be1" not yet present on node "kind-control-plane", loading...


In [63]:
%%bash
cd ml-intermediate
argo submit -n argo training/argo.yaml

Name:                ml-intermediate-527nn
Namespace:           argo
ServiceAccount:      default
Status:              Pending
Created:             Sat Aug 27 22:37:08 -0500 (now)
Progress:            


In [64]:
%%bash
argo wait @latest -n argo

@latest Succeeded at 2022-08-27 22:39:00 -0500 CDT


In [65]:
%%bash
argo get @latest -n argo

Name:                ml-intermediate-527nn
Namespace:           argo
ServiceAccount:      default
Status:              Succeeded
Conditions:          
 PodRunning          False
 Completed           True
Created:             Sat Aug 27 22:37:08 -0500 (1 minute ago)
Started:             Sat Aug 27 22:37:08 -0500 (1 minute ago)
Finished:            Sat Aug 27 22:39:00 -0500 (1 second ago)
Duration:            1 minute 52 seconds
Progress:            5/5
ResourcesDuration:   2m30s*(1 cpu),2m30s*(100Mi memory)

STEP                      TEMPLATE  PODNAME                           DURATION  MESSAGE
 ✔ ml-intermediate-527nn  dag                                                     
 ├─✔ get                  run-task  ml-intermediate-527nn-2388056745  13s         
 ├─✔ petal-area           run-task  ml-intermediate-527nn-891858397   19s         
 ├─✔ sepal-area           run-task  ml-intermediate-527nn-3280603182  19s         
 ├─✔ join                 run-task  ml-intermediate-527nn-314181195

In [67]:
ls outputs/sample=False

get.parquet         model.pickle        petal_area.parquet
join.parquet        nb.html             sepal_area.parquet
